# **Úloha 2 – Vizualizace dat**

Analýza a vizualizace dat pomocí knihoven `matplotlib` a `plotly`. Budu pracovat s datasetem obsahujícím údaje o vnitrostátních letech aerolinky AirIndia od fiskálního roku 2014 do fiskálního roku 2023.


1.   Month – měsíc
2.   Departures – celkový počet odletů za daný měsíc
3.   Hours – počet nalétaných hodin za daný měsíc
4.   Kilometers – počet nalétaných kilometrů za daný měsíc (v milionech)
5.   Passengers – počet pasažérů za daný měsíc (v milionech)
6.   Passenger_kms – násobek pasažérů a nalétaných kilometrů za daný měsíc (v milionech)
7.   Available_seat_kms – násobek dostupných sedadel a nalétaných kilometrů za daný měsíc (v milionech)
8.   Occupied_seats_% – procento obsazených sedadel z celkového počtu dostupých sedadel za daný měsíc
9.   FYear – fiskální rok

In [ ]:
### Úprava dat do použitelné podoby

import pandas as pd

data = pd.read_csv("airindia_domestic.csv")
data = data.dropna()
data.columns = ["Month", "Departures", "Hours", "Kilometers", "Passengers", "Passenger_kms", "Available_seat_kms", "Occupied_seats_%", "FYear"]
data["Kilometers"] = data["Kilometers"]/1000
data["Passengers"] = data["Passengers"]/1000000
data["Passenger_kms"] = data["Passenger_kms"]/1000
data["Available_seat_kms"] = data["Available_seat_kms"]/1000
data["Month"] = data["Month"].str.cat(data["FYear"], sep=" ")

print(data)

In [ ]:
### Liniové subploty se základním přehledem

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

month = data["Month"]
years = np.unique([m[-4:] for m in month])

plt.figure(figsize=(18, 12))

plt.subplot(231)
plt.plot(month, data["Departures"])
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Celkový počet odletů", fontsize=15)

plt.subplot(232)
plt.plot(month, data["Hours"])
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Celkový počet hodin letu", fontsize=15)

plt.subplot(233)
plt.plot(month, data["Kilometers"])
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Celkový počet nalétaných kilometrů\n(v milionech)", fontsize=15)

plt.subplot(234)
plt.plot(month, data["Passengers"])
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Celkový počet pasažérů (v milionech)", fontsize=15)

plt.subplot(235)
plt.plot(month, data["Available_seat_kms"])
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Celkový počet dostupných sedadel\nna kilometr (v milionech)", fontsize=15)

plt.subplot(236)
plt.plot(month, data["Occupied_seats_%"])
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Procento obsazených sedadel", fontsize=15)
frmt = "%.0f%%"
yticks = mtick.FormatStrFormatter(frmt)
plt.gca().yaxis.set_major_formatter(yticks)

plt.subplots_adjust(hspace=0.3)

plt.suptitle("Přehled základních údajů v čase – aerolinky AirIndia\nFiskální roky 2014–2023", fontsize=18)
plt.show()


U všech těchto grafů můžeme pozorovat sezónní výkyvy, a jelikož všechny tyto údaje (kromě procenta obsazených sedadel) mezi sebou zjevně korelují, nastavají tyto sezónní výkyvy přibližně ve stejné měsíce.

Dále můžeme ve všech grafech pozorovat mírný růst až do fiskálního roku 2021 (duben 2020 až duben 2021), kdy nastal obrovský propad, zřejmě kvůli pandemii COVID-19. Druhý velký propad byl zaznamenán ve fiskálním roce 2022, kdy byla zavedena další sada protiepidemických opatření.

In [ ]:
### Histogram – obsazená sedadla

import matplotlib.pyplot as plt
import numpy as np

seats = data["Occupied_seats_%"]
seats = seats[seats != 0]

plt.hist(seats, bins=20, range=None, density=True, color="green")
plt.xlabel("Obsazená sedadla")
plt.ylabel("Četnost")
plt.title("Procento obsazených sedadel – histogram")
frmt = "%.0f%%"
xticks = mtick.FormatStrFormatter(frmt)
plt.gca().xaxis.set_major_formatter(xticks)

plt.show()

Z histogramu můžeme vidět, že za normální situace bývá nejčastěji obsazeno okolo 80 % veškerých sedadel. Do histogramu jsem nezahrnul nulové hodnoty, které nastaly během pandemie COVID-19, jelikož se nejedná o normální situaci.

In [ ]:
### Scatterplot Matrix – korelace

import plotly.figure_factory as ff
import pandas as pd
import numpy as np

corr_data = data[data.columns[1:8]]

fig = ff.create_scatterplotmatrix(corr_data, diag="histogram", height=1200, width=1200)
fig.update_layout(title="Korelace mezi údaji aerolinky AirIndia")

fig.show()

Pomocí `create_scatterplotmatrix()` z knihovny plotly jsme ověřili, že mezi údaji je skutečně jasná korelace. Jediný údaj, kde je korelační koeficient nižší, je `Occupied_seats_%`, tedy procento obsazených sedadel.

In [ ]:
### Sloupcový graf a groupby

import pandas as pd
import matplotlib.pyplot as plt

months = data["Month"].str[:3]
passengers_data = pd.DataFrame({"Month": months, "Passengers": data["Passengers"]})

month_order = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
passengers_data["Month"] = pd.Categorical(passengers_data["Month"], categories=month_order, ordered=True)

monthly_mean = passengers_data.groupby("Month")["Passengers"].mean()

plt.bar(monthly_mean.index, monthly_mean.values)
plt.title("Průměrný počet pasažérů v každém měsíci (v milionech)")
plt.show()

Pomocí funkce `groupby()` jsme spočítali průměrný počet pasažérů za každý měsíc za posledních 10 fiskálních let, následně jsme tyto výsledky vizualizovali pomocí sloupcového grafu.

Můžeme vidět, že nejvíce pasažérů společnosti AirIndia na území Indie cestuje v zimních měsících. Toto může být z části způsobeno Vánocemi (i když v Indii slaví Vánoce spíše menšina obyvatel) nebo Indickými festivaly Holi a Lohri, které se slaví v zimních měsících. Více lidí tím pádem cestuje za svými rodinami, aby společně oslavovali. Nejméně pasažérů cestuje v létě.

In [ ]:
### Vývoj průměrných údajů v čase

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

month = data["Month"]
years = np.unique([m[-4:] for m in month])

avg_duration = np.where(data["Departures"] == 0, 0, data["Hours"]/data["Departures"])
avg_on_board = np.where(data["Departures"] == 0, 0, data["Passengers"]/data["Departures"]*10**6)

plt.figure(figsize=(12, 4))

plt.subplot(121)
plt.plot(month, avg_duration)
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Průměrná délka vnitrostátního letu (v hodinách)", fontsize=12)

plt.subplot(122)
plt.plot(month, avg_on_board)
plt.xticks(np.arange(0, len(month), 12), years)
plt.title("Průměrný počet pasažérů na palubě letadla", fontsize=12)

plt.show()

Když potřebujeme zjistit určité údaje, které v původní datové sadě nejsou obsaženy, můžeme si je spočítat z údajů, které máme k dispozici.

V těchto liniových grafech je vizualizovaný vývoj průměrné délky vnitrostátního letu a vývoj průměrného počtu pasažérů na palubě letadla během posledních 10 fiskálních let. Můžeme vidět, že lety jsou rok od roku průměrně o něco delší, zatímco průměrný počet pasažérů na palubě značně kolísá, ale v poslední době má spíše rostoucí trend.